# OpenSearch 벡터 스토어 및 Amazon Bedrock를 사용한 RAG 
>이 노트북은,
> - SageMaker Studio* **`Data Science 3.0`** kernel 및 ml.t3.medium 인스턴스에서 테스트 되었습니다.
> - SageMaker Notebook **`conda_python3`** 에서 테스트 되었습니다.

---
### 중요
- 이 노트북은 Anthropic 의 Claude-v2 모델 접근 가능한 분만 실행 가능합니다. 
- 접근이 안되시는 분은 노트북의 코드와 결과 만을 확인 하시면 좋겠습니다.
- 만일 실행시에는 **"과금"** 이 발생이 되는 부분 유념 해주시기 바랍니다.

### 선수조건
- 임베딩 모델의 세이지 메이커 엔드포인트가 액티브 된 상태를 가정 합니다.
    - 세이지 메이커 엔드포인트에 배포하기 위해서는 아래 노트북을 실행하시고, Endpoint Name 만을 복사 하시면 됩니다.
    - [KoSIMCSERoberta Embedding Model 배포](https://github.com/gonsoomoon-ml/Kor-LLM-On-SageMaker/blob/main/1-Lab01-Deploy-LLM/4.Kor-Embedding-Model.ipynb)
    - SageMaker Endpoint 에 대해서는 공식 개발자 문서를 참조하세요 --> [Create your endpoint and deploy your model](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-deployment.html)
- 오픈 서치 서비스가 액티브 된 상태를 가정 합니다.
---

OpenSearch는 대규모 데이터셋에 대한 유사도 검색을 위한 강력한 엔진입니다. Amazon OpenSearch Service를 통해 쉽게 클라우드 환경에서도 이용할 수 있습니다. 이와 함께 Vector Store를 사용하면 고차원 벡터 데이터를 효율적으로 저장하고 빠르게 검색할 수 있어, 복잡한 자연어 처리 작업을 더욱 간편하게 수행할 수 있습니다.


In [1]:
import boto3
region = boto3.Session().region_name
opensearch = boto3.client('opensearch', region)

%store -r opensearch_user_id opensearch_user_password domain_name opensearch_domain_endpoint new_index_name

try:
    opensearch_user_id
    opensearch_user_password
    domain_name
    opensearch_domain_endpoint
   
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] Run 00_setup notebook first or Create Your Own OpenSearch Domain")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [2]:
%load_ext autoreload
%autoreload 2

import sys, os
module_path = ".."
sys.path.append(os.path.abspath(module_path))

# 1. Bedrock Client 생성

In [42]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

bedrock = boto3.client(service_name='bedrock')
model_list = bedrock.list_foundation_models()
result = [(fm_list["modelName"], fm_list["modelId"]) for fm_list in model_list["modelSummaries"] if fm_list['inferenceTypesSupported'] == ['ON_DEMAND']]
pprint(result)

Create new client
  Using region: None
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-west-2.amazonaws.com)
[('Titan Text Large', 'amazon.titan-tg1-large'),
 ('Titan Text Embeddings v2', 'amazon.titan-embed-g1-text-02'),
 ('Titan Text G1 - Lite', 'amazon.titan-text-lite-v1'),
 ('Titan Text G1 - Express', 'amazon.titan-text-express-v1'),
 ('Titan Embeddings G1 - Text', 'amazon.titan-embed-text-v1'),
 ('Titan Multimodal Embeddings G1', 'amazon.titan-embed-image-v1'),
 ('Titan Image Generator G1', 'amazon.titan-image-generator-v1'),
 ('SDXL 0.8', 'stability.stable-diffusion-xl'),
 ('SDXL 0.8', 'stability.stable-diffusion-xl-v0'),
 ('SDXL 1.0', 'stability.stable-diffusion-xl-v1'),
 ('J2 Grande Instruct', 'ai21.j2-grande-instruct'),
 ('J2 Jumbo Instruct', 'ai21.j2-jumbo-instruct'),
 ('Jurassic-2 Mid', 'ai21.j2-mid'),
 ('Jurassic-2 Mid', 'ai21.j2-mid-v1'),
 ('Jurassic-2 Ultra', 'ai21.j2-ultra'),
 ('Jurassic-2 Ultra', 'ai21.j2-ultra

# 2. Titan Embedding 및 LLM 인 Claude-v2 모델 로딩

## LLM 로딩 (Claude-v2-1)

In [4]:
from langchain.llms.bedrock import Bedrock
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [5]:
# - create the Anthropic Model
llm_text = Bedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-V2-1"),
    client=boto3_bedrock,
    model_kwargs={
        "max_tokens_to_sample": 512
    },
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)
llm_text

Bedrock(client=<botocore.client.BedrockRuntime object at 0x7f22202dc670>, model_id='anthropic.claude-v2:1', model_kwargs={'max_tokens_to_sample': 512}, streaming=True, callbacks=[<langchain_core.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x7f22226d9a80>])

## Embedding 모델 선택

In [6]:
from utils.rag import KoSimCSERobertaContentHandler, SagemakerEndpointEmbeddingsJumpStart

def get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name=None):
    if is_bedrock_embeddings:

        # We will be using the Titan Embeddings Model to generate our Embeddings.
        from langchain.embeddings import BedrockEmbeddings

        llm_emb = BedrockEmbeddings(
            client=boto3_bedrock,
            model_id=bedrock_info.get_model_id(
                model_name="Titan-Embeddings-G1"
            )
        )

        print("Bedrock Embeddings Model Loaded")
        
    elif is_KoSimCSERobert:
        LLMEmbHandler = KoSimCSERobertaContentHandler()
        endpoint_name_emb = endpont_name
        llm_emb = SagemakerEndpointEmbeddingsJumpStart(
            endpoint_name=endpoint_name_emb,
            region_name=aws_region,
            content_handler=LLMEmbHandler,
        )        
        print("KoSimCSERobert Embeddings Model Loaded")
    else:
        llm_emb = None
        print("No Embedding Model Selected")
    
    return llm_emb

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


#### [중요] is_KoSimCSERobert == True 일시에 endpoint_name 을 꼭 넣어 주세요.

In [7]:
is_bedrock_embeddings = True
is_KoSimCSERobert = False

aws_region = os.environ.get("AWS_DEFAULT_REGION", None)

##############################
# Parameters for is_KoSimCSERobert
##############################
if is_KoSimCSERobert: endpont_name = "<endpoint-name>"
else: endpont_name = None
##############################

llm_emb = get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name)     

Bedrock Embeddings Model Loaded


# 3. 데이터 준비


#### `02_1_rag_opensearch_lexical_kr.ipynb`: 에서 데이터 준비 및 Index 생성을 해야 합니다.

# 4. OpenSearch 벡터 Indexer 생성
### 선수 조건
- 아래의 링크를 참조해서 OpenSearch Service 를 생성하고, opensearch_domain_endpoint, http_auth 를 복사해서, 아래 셀의 내용을 대체 하세요.
    - [OpenSearch 생성 가이드](https://github.com/gonsoomoon-ml/Kor-LLM-On-SageMaker/blob/main/2-Lab02-QA-with-RAG/4.rag-fsi-data-workshop/TASK-4_OpenSearch_Creation_and_Vector_Insertion.ipynb)
- 랭체인 오프서처 참고 자료
    - [Langchain Opensearch](https://python.langchain.com/docs/integrations/vectorstores/opensearch)

#### [중요] 아래에 aws parameter store 에 아래 인증정보가 먼저 입력되어 있어야 합니다.
- 20_applications/02_qa_chatbot/01_preprocess_docs/01_parameter_store_example.ipynb 참고

In [8]:
http_auth = (opensearch_user_id, opensearch_user_password) # Master username, Master password
index_name = "genai-demo-index-v1-with-tokenizer"

## 인덱싱된 벡터 확인

- [langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.html)

In [43]:
from langchain.vectorstores import OpenSearchVectorSearch

vector_db = OpenSearchVectorSearch(
    index_name=index_name,
    opensearch_url=opensearch_domain_endpoint,
    embedding_function=llm_emb,
    http_auth=http_auth, # http_auth
    is_aoss =False,
    engine="faiss",
    space_type="l2"
)

In [44]:
query='보험 약관'

In [45]:
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

wrapper_store = VectorStoreIndexWrapper(vectorstore=vector_db) 
answer = wrapper_store.query(question=query, llm=llm_text)

 Based on the information provided, here are the key points about 보험 약관 (insurance policy terms and conditions):

1. 보험약관은 보험계약의 내용과 조건 등을 미리 정한 계약조항으로, 보험계약자와 보험회사의 권리와 의무를 규정합니다. 

2. 보험약관에는 청약철회, 계약취소, 보험금 지급 등 중요사항이 포함되어 있습니다.

3. 주계약 약관은 기본계약 내용 등 공통 사항을 다룹니다.

4. 특약(특별약관)은 선택가입한 추가 보장에 대한 내용을 다룹니다.

5. 약관 이용 안내, 약관 요약서, 보험계약 일반사항 등을 통해 약관의 주요 내용을 쉽게 확인할 수 있습니다.

6. 특약 색인, 약관에서 인용된 법령 내용, 보험용어 해설 등도 약관 이용에 도움이 됩니다.

In summary, 보험약관 stipulates the rights and obligations of the policyholder and insurance company regarding the insurance contract. It is important to review the terms and conditions, especially sections on cancellation, claims, and other key details. Resources are available to help easily understand the policy terms.

## OpenSearch Dashboard 통한 Query 및 레코드 확인
- OpenSearch Dashboards URL 은 Amazon OpenSearch Servie 콘솔에 가시면 확인할 수 있습니다.
OpenSearch DevTools Console DSL 방법
- [OpenSearch_Query_DSL](https://opensearch.org/docs/latest/query-dsl/index/)

![record_opensearch](img/record_opensearch.jpg)

# 5.오픈 서치에 "유사 서치" 검색
- query 를 제공해서 실제로 유사한 내용이 검색이 되는지를 확인 합니다.



- similarity_search_with_score API 정보
    - [API: similarity_search_with_score](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.html#langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.similarity_search)

In [13]:
from langchain.chains.question_answering import load_qa_chain
from utils.opensearch import opensearch_utils

In [14]:
def get_semantic_similar_docs(**kwargs):

    search_types = ["approximate_search", "script_scoring", "painless_scripting"]
    space_types = ["l2", "l1", "linf", "cosinesimil", "innerproduct", "hammingbit"]

    assert "vector_db" in kwargs, "Check your vector_db"
    assert "query" in kwargs, "Check your query"
    assert kwargs.get("search_type", "approximate_search") in search_types, f'Check your search_type: {search_types}'
    assert kwargs.get("space_type", "l2") in space_types, f'Check your space_type: {space_types}'

    results = kwargs["vector_db"].similarity_search_with_score(
            query=kwargs["query"],
            k=kwargs.get("k", 5),
            search_type=kwargs.get("search_type", "approximate_search"),
            space_type=kwargs.get("space_type", "l2"),
            #boolean_filter=kwargs.get("boolean_filter", {}),        
            boolean_filter=opensearch_utils.get_filter(
                filter=kwargs.get("boolean_filter", [])
            ),        
            # fetch_k=3,
        )

    print ("\nsemantic search args: ")
    pprint ({
        "k": kwargs.get("k", 5),
        "search_type": kwargs.get("search_type", "approximate_search"),
        "space_type": kwargs.get("space_type", "l2"),
        "boolean_filter": opensearch_utils.get_filter(filter=kwargs.get("boolean_filter", []))
    })
    
    if kwargs.get("hybrid", False) and results:            
        max_score = results[0][1]
        new_results = []
        for doc in results:
            nomalized_score = float(doc[1]/max_score)
            new_results.append((doc[0], nomalized_score))
        results = copy.deepcopy(new_results)

    return results

## Filter 없이 실행. 

In [17]:
query = "계약 전 알릴 의무는 무엇인가요"
#query = "타기관OTP 등록 방법 알려주세요"

pre_similar_doc = get_semantic_similar_docs(
    vector_db=vector_db,
    query=query,
    k=3
)
opensearch_utils.opensearch_pretty_print_documents(pre_similar_doc)


semantic search args: 
{'boolean_filter': {'bool': {'filter': []}},
 'k': 3,
 'search_type': 'approximate_search',
 'space_type': 'l2'}

Score: 0.0063765203
Metadata:
Type: 간병치매보험_20240101~
Source: 한화생명 The실속있는 간병치매보험_20240101~.pdf
--------------------------------------------------

Score: 0.0063757636
Metadata:
Type: 간병치매보험_20231209~_1
Source: 한화생명 The실속있는 간병치매보험_20231209~_1.pdf
--------------------------------------------------

Score: 0.006014372
Metadata:
Type: 일시납종신보험_20240101~
Source: 한화생명 The스마트한 일시납종신보험_20240101~.pdf
--------------------------------------------------


## metadata의 type, source 에 필터를 걸어서 검색

In [28]:
filter01 = '일시납종신보험_20240101~'
filter02 = '한화생명 The스마트한 일시납종신보험_20240101~.pdf'

boolean_filter = [
    {"term": {"metadata.type": filter01}},
    {"term": {"metadata.source": filter02}},
]

pre_similar_doc = get_semantic_similar_docs(
    vector_db=vector_db,
    query=query,
    boolean_filter=boolean_filter,
    k=3
)
opensearch_utils.opensearch_pretty_print_documents(pre_similar_doc)


semantic search args: 
{'boolean_filter': {'bool': {'filter': [{'term': {'metadata.type': '일시납종신보험_20240101~'}},
                                        {'term': {'metadata.source': '한화생명 '
                                                                     'The스마트한 '
                                                                     '일시납종신보험_20240101~.pdf'}}]}},
 'k': 3,
 'search_type': 'approximate_search',
 'space_type': 'l2'}

Score: 0.006014372
Metadata:
Type: 일시납종신보험_20240101~
Source: 한화생명 The스마트한 일시납종신보험_20240101~.pdf
--------------------------------------------------

Score: 0.0058958996
Metadata:
Type: 일시납종신보험_20240101~
Source: 한화생명 The스마트한 일시납종신보험_20240101~.pdf
--------------------------------------------------

Score: 0.0051139095
Metadata:
Type: 일시납종신보험_20240101~
Source: 한화생명 The스마트한 일시납종신보험_20240101~.pdf
-----------------------

# 6.사용자 정의 가능한 옵션
이제 벡터 저장소가 준비되었으므로 질문을 시작할 수 있습니다.

Vector Store를 둘러싸서 LLM 입력을 받는 LangChain에서 제공하는 래퍼를 사용할 수 있습니다.
이 래퍼는 뒤에서 다음 단계를 수행합니다.
- 질문을 입력합니다.
- 질문 임베딩 생성
- 관련 문서 가져오기
- 프롬프트에 문서와 질문을 채워 넣습니다.
- 프롬프트로 모델을 호출하고 사람이 읽을 수 있는 방식으로 답변을 생성합니다.

위 시나리오에서는 질문에 대한 상황 인식 답변을 빠르고 쉽게 얻을 수 있는 방법을 탐색했습니다. 이제 문서를 가져오는 방법을 사용자 정의할 수 있는 [RetrievalQA](https://python.langchain.com/en/latest/modules/chains/index_examples/Vector_db_qa.html)의 도움으로 더 사용자 정의 가능한 옵션을 살펴보겠습니다. `chain_type` 매개변수를 사용하여 프롬프트에 추가해야 합니다. 또한 검색해야 하는 관련 문서 수를 제어하려면 아래 셀에서 'k' 매개변수를 변경하여 다른 출력을 확인하세요. 많은 시나리오에서 LLM이 답변을 생성하는 데 사용한 소스 문서가 무엇인지 알고 싶을 수 있습니다. LLM 프롬프트의 컨텍스트에 추가된 문서를 반환하는 `return_source_documents`를 사용하여 출력에서 ​​해당 문서를 가져올 수 있습니다. 'RetrievalQA'를 사용하면 모델에 특정한 사용자 정의 [프롬프트 템플릿](https://python.langchain.com/en/latest/modules/prompts/prompt_templates/getting_started.html)을 제공할 수도 있습니다.

참고: 이 예에서는 Amazon Bedrock에서 LLM으로 Anthropic Claude를 사용하고 있습니다. 이 특정 모델은 입력이 'Human:' 아래에 제공되고 모델이 'Assistant:' 다음에 출력을 생성하도록 요청되는 경우 가장 잘 수행됩니다. 아래 셀에는 LLM이 기본 상태를 유지하고 컨텍스트 외부에서 응답하지 않도록 프롬프트를 제어하는 ​​방법의 예가 나와 있습니다.

### [TIP] Prompt의 instruction의 경우 한글보다 **영어**로 했을 때 더 좋은 결과를 얻을 수 있습니다.

### [[REF] Using langchain for Question Answering on Own Data](https://medium.com/@onkarmishra/using-langchain-for-question-answering-on-own-data-3af0a82789ed)

In [29]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from utils.rag import run_RetrievalQA, show_context_used

In [30]:
# prompt_template = """
# \n\nHuman: 다음 문맥의 Information을 사용하여 고객 서비스 센터 직원처럼, 마지막 질문에 대한 목차 형식으로 답변을 제공하세요. 응답을 모르면 모른다고 말하고 응답을 만들려고 하지 마세요.

# {context}

# Question: {question}
# \n\nAssistant:"""

prompt_template = """
\n\nHuman: Use the following pieces of context to provide a concise answer to the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}

\n\nAssistant:"""


PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

def run_RetrievalQA(**kwargs):

    chain_types = ["stuff", "map_reduce", "refine"]

    assert "llm" in kwargs, "Check your llm"
    assert "query" in kwargs, "Check your query"
    assert "prompt" in kwargs, "Check your prompt"
    assert "vector_db" in kwargs, "Check your vector_db"
    assert kwargs.get("chain_type", "stuff") in chain_types, f'Check your chain_type, {chain_types}'

    qa = RetrievalQA.from_chain_type(
        llm=kwargs["llm"],
        chain_type=kwargs.get("chain_type", "stuff"),
        retriever=kwargs["vector_db"].as_retriever(
            search_type="similarity",
            search_kwargs={
                "k": kwargs.get("k", 5),
                "boolean_filter": opensearch_utils.get_filter(
                    filter=kwargs.get("boolean_filter", [])
                ),
            }
        ),
        return_source_documents=True,
        chain_type_kwargs={
            "prompt": kwargs["prompt"],
            "verbose": kwargs.get("verbose", False),
        },
        verbose=kwargs.get("verbose", False)
    )

    return qa(kwargs["query"])

In [31]:
from utils.rag import run_RetrievalQA

In [32]:
# query = "홈페이지 탈퇴하는방법 알려줘"

result = run_RetrievalQA(
    query=query,
    llm=llm_text,
    prompt=PROMPT,
    vector_db=vector_db,
    verbose=False,
    k=3
)

print("##################################")
print("query: ", query)
print("##################################")
print_ww(result['result'])

 문서에서 찾은 내용을 참고해서 답변드리겠습니다.

계약 전 알릴 의무란 청약할 때(진단계약의 경우 건강진단할 때) 청약서에서 질문한 사항에 대해 계약자나 피보험자가 알고 있는 사실을 반드시 사실대로 알려야 하는 의무를 말합니다. 이를 위반할 경우 보험계약의 해지나 보험금 부지급 등의 불이익을 받을 수 있습니다.##################################
query:  계약 전 알릴 의무는 무엇인가요
##################################
 문서에서 찾은 내용을 참고해서 답변드리겠습니다.

계약 전 알릴 의무란 청약할 때(진단계약의 경우 건강진단할 때) 청약서에서 질문한 사항에 대해 계약자나 피보험자가 알고 있는 사실을 반드시 사실대로 알려야 하는 의무를 말합니다.
이를 위반할 경우 보험계약의 해지나 보험금 부지급 등의 불이익을 받을 수 있습니다.


In [33]:
show_context_used(result['source_documents'])

-----------------------------------------------
1. Chunk: 1873 Characters
-----------------------------------------------
다.
제 3 관 계약자의 계약 전 알릴 의무 등
제 14 조 계약 전 알릴 의무
계약자 또는 피보험자는 청약할 때(진단계약의 경우에는 건강진단할 때를 말합니다) 청약서에서 질문한
사항에 대하여 알고 있는 사실을 반드시 사실대로 알려야(이하 ‘계약 전 알릴 의무’라 하며, 상법상 ‘고
지의무’와 같습니다)합니다. 다만, 진단계약에서 의료법 제3조(의료기관)의 규정에 따른 종합병원과 병원
에서 직장 또는 개인이 실시한 건강진단서 사본 등 건강상태를 판단할 수 있는 자료로 건강진단을 대신
할 수 있습니다.
【계약 전 알릴 의무】
상법 제651조(고지의무 위반으로 인한 계약해지)에서 정하고 있는 의무. 보험계약자나 피보험자는 청
약시에 보험회사가 질문한 중요한 사항에 대해 사실대로 알려야 하며, 위반시 보험계약의 해지 또는
보험금 부지급 등 불이익을 당할 수 있습니다.
제 15 조 계약 전 알릴 의무 위반의 효과
① 회사는 계약자 또는 피보험자가 제14조(계약 전 알릴 의무)에도 불구하고 고의 또는 중대한 과실로
중요한 사항에 대하여 사실과 다르게 알린 경우에는 회사가 별도로 정하는 방법에 따라 계약을 해지하
거나 보장을 제한할 수 있습니다. 그러나 다음 중 한 가지에 해당되는 때에는 계약을 해지하거나 보장
을 제한할 수 없습니다.
1. 회사가 계약 당시에 그 사실을 알았거나 과실로 인하여 알지 못하였을 때
2. 회사가 그 사실을 안 날부터 1개월 이상 지났거나 또는 보장개시일부터 보험금 지급사유가 발생하
지 않고 2년(진단계약의 경우 질병에 대하여는 1년)이 지났을 때
3. 계약을 체결한 날부터 3년이 지났을 때
4. 회사가 이 계약을 청약할 때 피보험자의 건강상태를 판단할 수 있는 기초자료(건강진단서 사본 등)
에 따라 승낙한 경우에 건강진단서 사본

In [36]:
prompt_template = """

Human: 너는 친절하게 정보를 제공하는 인공지능 비서야.
사용자가 질문하면 <content> tag 안에 있는 내용과 너가 알고 있는 지식을 더해서 답변을 해줘야 해.
사용자의 질문은 <question> tag 안에 있어.

<content>
{context}
</content>

<question>
{question}
</question>

<content> tag 안에 질문과 관련된 내용이 없으면 '사용자의 질문에 대한 내용을 문서에서 찾을 수는 없지만 제가 알고 있는 내용은' 이라는 서두와 함께 답변해 줘
<content> tag 안에 질문과 관련된 내용이 있으면 '문서에서 찾은 내용을 참고해서 답변드리겠습니다' 라는 서두와 함께 답변해줘 
"tag 에 따르면" 이라는 서두는 제외하고 답변해줘.
답변은 모두 공손하게 해줘

답변은 250 토큰 이내로 해줘

Assistant:"""
    

PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

In [37]:
filter01 = '간병치매보험_20231209~_1'
filter02 = '한화생명 The실속있는 간병치매보험_20231209~_1.pdf'


boolean_filter = [
    {"term": {"metadata.type": filter01}},
    # {"term": {"metadata.source": filter02}},
]

result = run_RetrievalQA(
    query=query,
    boolean_filter=boolean_filter,
    llm=llm_text,
    prompt=PROMPT,
    vector_db=vector_db,
    verbose=False,
    k=3
)

print("##################################")
print("query: ", query)
print("boolean_filter: ", boolean_filter)
print("##################################")
print_ww(result['result'])

 문서에서 찾은 내용을 참고해서 답변드리겠습니다.

계약 전 알릴 의무란 보험계약자나 피보험자가 보험계약 청약 시에 청약서에서 질문한 중요한 사항에 대해 사실대로 알려야 하는 의무를 말합니다. 이를 위반하면 보험계약 해지나 보험금 부지급 등의 불이익을 받을 수 있습니다. 알리지 않은 사항이 중요한 사항인지 여부와 고의 또는 과실 여부에 따라 위반 시 불이익이 다릅니다.##################################
query:  계약 전 알릴 의무는 무엇인가요
boolean_filter:  [{'term': {'metadata.type': '간병치매보험_20231209~_1'}}]
##################################
 문서에서 찾은 내용을 참고해서 답변드리겠습니다.

계약 전 알릴 의무란 보험계약자나 피보험자가 보험계약 청약 시에 청약서에서 질문한 중요한 사항에 대해 사실대로 알려야 하는 의무를 말합니다. 이를 위반하면 보험계약 해지나 보험금
부지급 등의 불이익을 받을 수 있습니다. 알리지 않은 사항이 중요한 사항인지 여부와 고의 또는 과실 여부에 따라 위반 시 불이익이 다릅니다.


In [38]:
show_context_used(result['source_documents'])

-----------------------------------------------
1. Chunk: 1873 Characters
-----------------------------------------------
다.
제 3 관 계약자의 계약 전 알릴 의무 등
제 14 조 계약 전 알릴 의무
계약자 또는 피보험자는 청약할 때(진단계약의 경우에는 건강진단할 때를 말합니다) 청약서에서 질문한
사항에 대하여 알고 있는 사실을 반드시 사실대로 알려야(이하 ‘계약 전 알릴 의무’라 하며, 상법상 ‘고
지의무’와 같습니다)합니다. 다만, 진단계약에서 의료법 제3조(의료기관)의 규정에 따른 종합병원과 병원
에서 직장 또는 개인이 실시한 건강진단서 사본 등 건강상태를 판단할 수 있는 자료로 건강진단을 대신
할 수 있습니다.
【계약 전 알릴 의무】
상법 제651조(고지의무 위반으로 인한 계약해지)에서 정하고 있는 의무. 보험계약자나 피보험자는 청
약시에 보험회사가 질문한 중요한 사항에 대해 사실대로 알려야 하며, 위반시 보험계약의 해지 또는
보험금 부지급 등 불이익을 당할 수 있습니다.
제 15 조 계약 전 알릴 의무 위반의 효과
① 회사는 계약자 또는 피보험자가 제14조(계약 전 알릴 의무)에도 불구하고 고의 또는 중대한 과실로
중요한 사항에 대하여 사실과 다르게 알린 경우에는 회사가 별도로 정하는 방법에 따라 계약을 해지하
거나 보장을 제한할 수 있습니다. 그러나 다음 중 한 가지에 해당되는 때에는 계약을 해지하거나 보장
을 제한할 수 없습니다.
1. 회사가 계약 당시에 그 사실을 알았거나 과실로 인하여 알지 못하였을 때
2. 회사가 그 사실을 안 날부터 1개월 이상 지났거나 또는 보장개시일부터 보험금 지급사유가 발생하
지 않고 2년(진단계약의 경우 질병에 대하여는 1년)이 지났을 때
3. 계약을 체결한 날부터 3년이 지났을 때
4. 회사가 이 계약을 청약할 때 피보험자의 건강상태를 판단할 수 있는 기초자료(건강진단서 사본 등)
에 따라 승낙한 경우에 건강진단서 사본

In [40]:
filter01 = '간병치매보험_20240101~'
filter02 = '한화생명 The실속있는 간병치매보험_20240101~.pdf'

boolean_filter = [
    {"term": {"metadata.type": filter01}},
    {"term": {"metadata.source": filter02}},
]

result = run_RetrievalQA(
    query=query,
    boolean_filter=boolean_filter,
    llm=llm_text,
    prompt=PROMPT,
    vector_db=vector_db,
    verbose=True,
    k=3
)

print("##################################")
print("query: ", query)
print("boolean_filter: ", boolean_filter)
print("##################################")
print_ww(result['result'])



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:


Human: 너는 친절하게 정보를 제공하는 인공지능 비서야.
사용자가 질문하면 <content> tag 안에 있는 내용과 너가 알고 있는 지식을 더해서 답변을 해줘야 해.
사용자의 질문은 <question> tag 안에 있어.

<content>
다.
제 3 관 계약자의 계약 전 알릴 의무 등
제 14 조 계약 전 알릴 의무
계약자 또는 피보험자는 청약할 때(진단계약의 경우에는 건강진단할 때를 말합니다) 청약서에서 질문한
사항에 대하여 알고 있는 사실을 반드시 사실대로 알려야(이하 ‘계약 전 알릴 의무’라 하며, 상법상 ‘고
지의무’와 같습니다)합니다. 다만, 진단계약에서 의료법 제3조(의료기관)의 규정에 따른 종합병원과 병원
에서 직장 또는 개인이 실시한 건강진단서 사본 등 건강상태를 판단할 수 있는 자료로 건강진단을 대신
할 수 있습니다.
【계약 전 알릴 의무】
상법 제651조(고지의무 위반으로 인한 계약해지)에서 정하고 있는 의무. 보험계약자나 피보험자는 청
약시에 보험회사가 질문한 중요한 사항에 대해 사실대로 알려야 하며, 위반시 보험계약의 해지 또는
보험금 부지급 등 불이익을 당할 수 있습니다.
제 15 조 계약 전 알릴 의무 위반의 효과
① 회사는 계약자 또는 피보험자가 제14조(계약 전 알릴 의무)에도 불구하고 고의 또는 중대한 과실로
중요한 사항에 대하여 사실과 다르게 알린 경우에는 회사가 별도로 정하는 방법에 따라 계약을 해지하
거나 보장을 제한할 수 있습니다. 그러나 다음 중 한 가지에 해당되는 때에는 계약을 해지하거나 보장
을 제한할 수 없습니다.
1. 회사가 계약 당시에 그 사실을 알았거나 과실로 인하여 알지 못하였을 때
2. 회사가 그 사실을 안 날부터 1개월 이상 지났거나 또는 

In [41]:
show_context_used(result['source_documents'])

-----------------------------------------------
1. Chunk: 1873 Characters
-----------------------------------------------
다.
제 3 관 계약자의 계약 전 알릴 의무 등
제 14 조 계약 전 알릴 의무
계약자 또는 피보험자는 청약할 때(진단계약의 경우에는 건강진단할 때를 말합니다) 청약서에서 질문한
사항에 대하여 알고 있는 사실을 반드시 사실대로 알려야(이하 ‘계약 전 알릴 의무’라 하며, 상법상 ‘고
지의무’와 같습니다)합니다. 다만, 진단계약에서 의료법 제3조(의료기관)의 규정에 따른 종합병원과 병원
에서 직장 또는 개인이 실시한 건강진단서 사본 등 건강상태를 판단할 수 있는 자료로 건강진단을 대신
할 수 있습니다.
【계약 전 알릴 의무】
상법 제651조(고지의무 위반으로 인한 계약해지)에서 정하고 있는 의무. 보험계약자나 피보험자는 청
약시에 보험회사가 질문한 중요한 사항에 대해 사실대로 알려야 하며, 위반시 보험계약의 해지 또는
보험금 부지급 등 불이익을 당할 수 있습니다.
제 15 조 계약 전 알릴 의무 위반의 효과
① 회사는 계약자 또는 피보험자가 제14조(계약 전 알릴 의무)에도 불구하고 고의 또는 중대한 과실로
중요한 사항에 대하여 사실과 다르게 알린 경우에는 회사가 별도로 정하는 방법에 따라 계약을 해지하
거나 보장을 제한할 수 있습니다. 그러나 다음 중 한 가지에 해당되는 때에는 계약을 해지하거나 보장
을 제한할 수 없습니다.
1. 회사가 계약 당시에 그 사실을 알았거나 과실로 인하여 알지 못하였을 때
2. 회사가 그 사실을 안 날부터 1개월 이상 지났거나 또는 보장개시일부터 보험금 지급사유가 발생하
지 않고 2년(진단계약의 경우 질병에 대하여는 1년)이 지났을 때
3. 계약을 체결한 날부터 3년이 지났을 때
4. 회사가 이 계약을 청약할 때 피보험자의 건강상태를 판단할 수 있는 기초자료(건강진단서 사본 등)
에 따라 승낙한 경우에 건강진단서 사본